# NLVR Experiment by using pre-trained VL-BART and VL-T5

## Download code and install dependencies

In [ ]:
!git clone https://github.com/j-min/VL-T5


fatal: destination path 'VL-T5' already exists and is not an empty directory.


In [ ]:
!ls

sample_data  VL-T5


In [ ]:
!pip uninstall param -y # to resolve name conflict with src.param.py
!pip install -r VL-T5/requirements.txt
!python download_backbones.py

Found existing installation: param 1.12.0
Uninstalling param-1.12.0:
  Successfully uninstalled param-1.12.0
  Cloning git://github.com/j-min/language-evaluation (to revision master) to /tmp/pip-req-build-hsi826ga
  Running command git clone -q git://github.com/j-min/language-evaluation /tmp/pip-req-build-hsi826ga
     |████████████████████████████████| 748.8 MB 19 kB/s 
     |████████████████████████████████| 1.8 MB 41.6 MB/s 
     |████████████████████████████████| 1.2 MB 39.0 MB/s 
     |████████████████████████████████| 1.7 MB 40.9 MB/s 
     |████████████████████████████████| 90 kB 10.1 MB/s 
     |████████████████████████████████| 5.9 MB 21.9 MB/s 
     |████████████████████████████████| 2.9 MB 37.2 MB/s 
     |████████████████████████████████| 895 kB 53.6 MB/s 
     |████████████████████████████████| 142 kB 53.9 MB/s 
     |████████████████████████████████| 180 kB 50.7 MB/s 
     |████████████████████████████████| 97 kB 6.9 MB/s 
     |████████████████████████████████| 63 kB 1.9

## Download the pretrained checkpoint

In [ ]:
import gdown

In [ ]:
!mkdir -p  /content/VL-T5/VL-T5/snap/pretrain/VLT5
!mkdir -p  /content/VL-T5/VL-T5/snap/pretrain/VLBart
#!mkdir -p snap/vcr_pretrain/VLBart
#!mkdir -p snap/vcr_pretrain/VLT5

In [ ]:
gdown.download('https://drive.google.com/uc?id=100qajGncE_vc4bfjVxxICwz3dwiAxbIZ', '/content/VL-T5/VL-T5/snap/pretrain/VLT5/Epoch30.pth', quiet=False)
gdown.download('https://drive.google.com/uc?id=1fTKGCBfMe2eJ_rivPQu0YkNJTNdv_0aq', '/content/VL-T5/VL-T5/snap/pretrain/VLBart/Epoch30.pth', quiet=False)
#gdown.download('https://drive.google.com/uc?id=1jXyMnJJqwrIrHk62WB_ye0dPxBNvIoe5', 'snap/vcr_pretrain/VLT5/Epoch20.pth', quiet=False)
#gdown.download('https://drive.google.com/uc?id=1urZFOZTpXuMbU_Q697fwGdmg36an7LDP', 'snap/vcr_pretrain/VLBart/Epoch20.pth', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=100qajGncE_vc4bfjVxxICwz3dwiAxbIZ
To: /content/VL-T5/VL-T5/snap/pretrain/VLT5/Epoch30.pth
100%|██████████| 898M/898M [00:08<00:00, 109MB/s]
Downloading...
From: https://drive.google.com/uc?id=1fTKGCBfMe2eJ_rivPQu0YkNJTNdv_0aq
To: /content/VL-T5/VL-T5/snap/pretrain/VLBart/Epoch30.pth
100%|██████████| 565M/565M [00:12<00:00, 46.0MB/s]


'/content/VL-T5/VL-T5/snap/pretrain/VLBart/Epoch30.pth'

Download datasets

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls /content/gdrive/MyDrive/datasets

Mounted at /content/gdrive
COCO  flickr30k  GQA  lxmert  nlvr  RefCOCO  VCR  VG  vqa


In [ ]:
!cp -r '/content/gdrive/MyDrive/datasets' '/content/VL-T5/'
!ls '/content/VL-T5/datasets'

COCO  flickr30k  GQA  lxmert  nlvr  RefCOCO  VCR  VG  vqa


In [ ]:
!ls '/content/VL-T5/datasets/nlvr'

features  test.json  train.json  valid.json


In [ ]:
cd /content/VL-T5/VL-T5/src

/content/VL-T5/VL-T5/src


In [ ]:
!ls

caption_data.py   modeling_bart.py    pretrain_data.py	    tokenization.py
caption_model.py  modeling_t5.py      pretrain_model.py     trainer_base.py
caption.py	  multitask_data.py   pretrain.py	    utils.py
dist_utils.py	  multitask_model.py  pretrain_vcr_data.py  vcr_data.py
gqa_data.py	  multitask.py	      pretrain_vcr.py	    vcr_model.py
gqa_model.py	  nlvr_data.py	      qa_answer_table.py    vcr.py
gqa.py		  nlvr_model.py       refcoco_data.py	    vqa_data.py
mmt_data.py	  nlvr.py	      refcoco_model.py	    vqa_model.py
mmt_model.py	  param.py	      refcoco.py	    vqa.py
mmt.py		  preprocess.py       refcoco_utils.py


In [ ]:
!ls  /content/VL-T5/VL-T5/snap/pretrain/VLT5

Epoch30.pth


In [ ]:
!ls  /content/VL-T5/VL-T5/snap/pretrain/VLBart

Epoch30.pth


# VL-T5 on NLVR

In [ ]:
cd /content/VL-T5/VL-T5/src

/content/VL-T5/VL-T5/src


In [ ]:
from nlvr import Trainer
from nlvr_data import get_loader
from param import parse_args

In [ ]:
vl_t5_args = parse_args(
    parse=False,
    backbone='t5-base',
    load='/content/VL-T5/VL-T5/snap/pretrain/VLT5/Epoch30',
    gpu = 0
)
valid_batch_size = 10

In [ ]:
print(f'Building test submit loader at GPU {0}')
vl_t5_test_loader = get_loader(
  vl_t5_args,
  split='test', mode='test', batch_size=valid_batch_size,
  gpu=vl_t5_args.gpu,
  workers=4,
)


Building test submit loader at GPU 0
Load 6967 data from split(s) test.
Data source:  test
# all sentences: 6967


In [ ]:
vl_t5_trainer = Trainer(
    vl_t5_args,
    train=False,
    test_loader=vl_t5_test_loader
)

Building Model at GPU 0
Model loaded from  /content/VL-T5/VL-T5/snap/pretrain/VLT5/Epoch30.pth
_IncompatibleKeys(missing_keys=[], unexpected_keys=['encoder.visual_embedding.layer_norm.weight'])
Model Launching at GPU 0
It took 0.6s


In [ ]:
vl_t5_acc_dict = vl_t5_trainer.evaluate(vl_t5_test_loader)

Prediction: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 697/697 [03:53<00:00,  2.98it/s]


In [ ]:
print("VL-T5")
print("overall: {}".format(round(100*vl_t5_acc_dict.get("accuracy"),1)))

VL-T5
overall: 50.1


# VL-BART on NLVR

In [ ]:
from nlvr import Trainer
from nlvr_data import get_loader
from param import parse_args

In [ ]:
vl_bart_args = parse_args(
    parse=False,
    backbone='facebook/bart-base',
    load='/content/VL-T5/VL-T5/snap/pretrain/VLBart/Epoch30',
    gpu = 0
)
valid_batch_size = 10

In [ ]:
print(f'Building test submit loader at GPU {0}')
vl_bart_test_loader = get_loader(
  vl_bart_args,
  split='test', mode='test', batch_size=valid_batch_size,
  gpu=vl_bart_args.gpu,
  workers=4,
)

Building test submit loader at GPU 0
Load 6967 data from split(s) test.
Data source:  test
# all sentences: 6967


In [ ]:
vl_bart_trainer = Trainer(
    vl_bart_args,
    train=False,
    test_loader=vl_bart_test_loader
)

Building Model at GPU 0
Vocab resize: 50265 -> 50465
Model loaded from  /content/VL-T5/VL-T5/snap/pretrain/VLBart/Epoch30.pth
_IncompatibleKeys(missing_keys=['model.encoder.visual_embedding.feat_embedding.1.weight', 'model.encoder.visual_embedding.feat_embedding.1.bias', 'model.encoder.visual_embedding.absolute_vis_pos_embedding.1.weight', 'model.encoder.visual_embedding.absolute_vis_pos_embedding.1.bias'], unexpected_keys=['model.encoder.visual_embedding.layer_norm.weight', 'model.encoder.visual_embedding.layer_norm.bias'])
Model Launching at GPU 0
It took 0.3s


In [ ]:
vl_bart_acc_dict = vl_bart_trainer.evaluate(vl_bart_test_loader)

Prediction: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 697/697 [01:43<00:00,  6.76it/s]


In [ ]:
print("VL-Bart")
print("overall: {}".format(round(100*vl_bart_acc_dict.get("accuracy"),1)))

VL-Bart
overall: 48.6
